In [3]:
import jsonlines

from config import TLCPaths

In [4]:
def get_data():
    with jsonlines.open(TLCPaths.project_data_path.joinpath(
            "TLC-validation-second-round_dump.jsonl")) as reader:
        for obj in reader:
            yield obj

In [5]:
annotations = list(get_data())

In [6]:
len(annotations) / 2  # in the ends its 832

53.0

In [ ]:
annotations1 = {}
id1 = 'TLC-validation-second-round-alon'
annotations2 = {}
id2 = 'TLC-validation-second-round-selin'
for annotation in annotations:
    name = annotation["_input_hash"]
    a_id = annotation["_annotator_id"]
    if a_id == id1:
        annotations1[name] = annotation
    elif a_id == id2:
        annotations2[name] = annotation
    else:
        print("unknown annotator id", a_id)#%%
import jsonlines

from config import TLCPaths

In [ ]:
def get_data():
    with jsonlines.open(TLCPaths.project_data_path.joinpath(
            "TLC-validation-second-round_dump.jsonl")) as reader:
        for obj in reader:
            yield obj

In [ ]:
annotations = list(get_data())

In [ ]:
len(annotations) / 2  # in the ends its 832

In [ ]:
annotations1 = {}
id1 = 'TLC-validation-second-round-alon'
annotations2 = {}
id2 = 'TLC-validation-second-round-selin'
for annotation in annotations:
    name = annotation["_input_hash"]
    a_id = annotation["_annotator_id"]
    if a_id == id1:
        annotations1[name] = annotation
    elif a_id == id2:
        annotations2[name] = annotation
    else:
        print("unknown annotator id", a_id)
    
    

In [ ]:
same = []
diff = []
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == ann2["answer"]:
        same.append(name)
    else:
        diff.append(name)
    

In [ ]:
print(len(same), len(diff))

In [ ]:
# treat one ignore as same with the other answer and treat both ignore as both rejected
same = []
diff = []
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == "ignore" and ann2["answer"] == "ignore":
        ann2['answer'] = "reject"
        ann1['answer'] = "reject"
        same.append(name)
    elif ann1["answer"] == ann2["answer"]:
        same.append(name)
    elif ann1["answer"] == "ignore" and not ann2["answer"] == "ignore":
        ann1['answer'] = ann2['answer']
        same.append(name)
    elif ann2["answer"] == "ignore" and not ann1["answer"] == "ignore":
        ann2['answer'] = ann1['answer']
        same.append(name)

    else:
        diff.append(name)

In [ ]:
print(len(same), len(diff))

In [ ]:
# get annotations that differ or both rejected
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == "reject" or ann2["answer"] == "reject":
        print(ann1["_input_hash"])
# 3 both rejected and 7 not same answer

In [ ]:
not_accepted1 = []
accepted1 = []
for name, ann in annotations1.items():
    if ann["answer"] != "accept":
        not_accepted1.append(name)
    else:
        accepted1.append(name)
print("not accepted: ",len(not_accepted1), "accepted: ", len(accepted1))
    

In [ ]:
not_accepted2 = []
accepted2 = []
for name, ann in annotations2.items():
    if ann["answer"] != "accept":
        not_accepted2.append(name)
    else:
        accepted2.append(name)
print("not accepted: ",len(not_accepted2), "accepted: ", len(accepted2))

In [ ]:
print("both accepted: ", len(set(accepted1).intersection(set(accepted2))))
print("both discarded: ", len(set(not_accepted1).intersection(set(not_accepted2))))

In [ ]:
print("accepted1 but not accepted2: ", len(set(accepted1).difference(set(accepted2))))
print("accepted2 but not accepted1: ", len(set(accepted2).difference(set(accepted1))))

print("not_accepted1 but not_accepted2: ", len(set(not_accepted1).difference(set(not_accepted2))))
print("not_accepted2 but not_accepted1: ", len(set(not_accepted2).difference(set(not_accepted1))))

In [ ]:
for name in diff:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    print("Answers: ", ann1["answer"],"-", ann2["answer"])
    start = ann1["spans"][0]["start"]
    end = ann1["spans"][0]["end"]
    print(ann1["text"][start:end], "-> ", ann1['html'])
    print()

In [ ]:
diff1 = [annotations1[name] for name in diff]
diff2 = [annotations2[name] for name in diff]

In [ ]:
diff1[0]

annotator agreement


In [ ]:
# calculate kappa score given that we treat ignore as same with the other answer and treat both ignore as both rejected
a1, a2 = [], []
for name in annotations1:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    a1.append(ann1["answer"])
    a2.append(ann2["answer"])
    
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(a1, a2)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(a1, a2))

In [ ]:
print(classification_report(a2, a1))

In [ ]:

import collections

def parse_classification_report(clfreport):
    """
    Parse a sklearn classification report into a dict keyed by class name
    and containing a tuple (precision, recall, fscore, support) for each class
    """
    lines = clfreport.split('\n')
    # Remove empty lines
    lines = list(filter(lambda l: not len(l.strip()) == 0, lines))
    lines=[s for s in lines if 'accuracy' not in s]

    # Starts with a header, then score for each class and finally an average
    header = lines[0]
    cls_lines = lines[1:-1]
    avg_line = lines[-1]

    assert header.split() == ['precision', 'recall', 'f1-score', 'support']
    assert avg_line.split()[1] == 'avg'

    # We cannot simply use split because class names can have spaces. So instead
    # figure the width of the class field by looking at the indentation of the
    # precision header
    cls_field_width = len(header) - len(header.lstrip())
    # Now, collect all the class names and score in a dict
    def parse_line(l):
        """Parse a line of classification_report"""
        cls_name = l[:cls_field_width].strip()
        precision, recall, fscore, support = l[cls_field_width:].split()
        precision = float(precision)
        recall = float(recall)
        fscore = float(fscore)
        support = int(support)
        return (cls_name, precision, recall, fscore, support)

    data = collections.OrderedDict()
    for l in cls_lines:
        ret = parse_line(l)
        cls_name = ret[0]
        scores = ret[1:]
        data[cls_name] = scores

    # average
    data['avg'] = parse_line(avg_line)[1:]

    return data

def report_to_latex_table(data):
    avg_split = False
    out = ""
    out += "\\begin{table}\n"
    out += "\\caption{Latex Table from Classification Report}\n"
    out += "\\label{table:classification:report}\n"
    out += "\\centering\n"
    out += "\\begin{tabular}{c | c c c r}\n"
    out += "Class & Precision & Recall & F-score & Support\\\\\n"
    out += "\midrule\n"
    for cls, scores in data.items():
        if 'micro' in cls:
            out += "\\midrule\n"
        out += cls + " & " + " & ".join([str(s) for s in scores])
        out += "\\\\\n"
    out += "\\end{tabular}\n"
    out += "\\end{table}"
    return out

print(report_to_latex_table(parse_classification_report(classification_report(a1, a2))))

In [9]:
same = []
diff = []
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == ann2["answer"]:
        same.append(name)
    else:
        diff.append(name)
    

In [10]:
print(len(same), len(diff))

43 10


In [26]:
# treat one ignore as same with the other answer and treat both ignore as both rejected
same = []
diff = []
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == "ignore" and ann2["answer"] == "ignore":
        ann2['answer'] = "reject"
        ann1['answer'] = "reject"
        same.append(name)
    elif ann1["answer"] == ann2["answer"]:
        same.append(name)
    elif ann1["answer"] == "ignore" and not ann2["answer"] == "ignore":
        ann1['answer'] = ann2['answer']
        same.append(name)
    elif ann2["answer"] == "ignore" and not ann1["answer"] == "ignore":
        ann2['answer'] = ann1['answer']
        same.append(name)

    else:
        diff.append(name)

In [27]:
print(len(same), len(diff))

46 7


In [37]:
# get annotations that differ or both rejected
for name in annotations2:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    if ann1["answer"] == "reject" or ann2["answer"] == "reject":
        print(ann1["_input_hash"])
# 3 both rejected and 7 not same answer

1836920489
1591926120
1001234244
247748062
167102995
1966492551
5270799
-988021557
-780350396
-255809126


In [28]:
not_accepted1 = []
accepted1 = []
for name, ann in annotations1.items():
    if ann["answer"] != "accept":
        not_accepted1.append(name)
    else:
        accepted1.append(name)
print("not accepted: ",len(not_accepted1), "accepted: ", len(accepted1))
    

not accepted:  7 accepted:  46


In [29]:
not_accepted2 = []
accepted2 = []
for name, ann in annotations2.items():
    if ann["answer"] != "accept":
        not_accepted2.append(name)
    else:
        accepted2.append(name)
print("not accepted: ",len(not_accepted2), "accepted: ", len(accepted2))

not accepted:  6 accepted:  47


In [30]:
print("both accepted: ", len(set(accepted1).intersection(set(accepted2))))
print("both discarded: ", len(set(not_accepted1).intersection(set(not_accepted2))))

both accepted:  43
both discarded:  3


In [31]:
print("accepted1 but not accepted2: ", len(set(accepted1).difference(set(accepted2))))
print("accepted2 but not accepted1: ", len(set(accepted2).difference(set(accepted1))))

print("not_accepted1 but not_accepted2: ", len(set(not_accepted1).difference(set(not_accepted2))))
print("not_accepted2 but not_accepted1: ", len(set(not_accepted2).difference(set(not_accepted1))))

accepted1 but not accepted2:  3
accepted2 but not accepted1:  4
not_accepted1 but not_accepted2:  4
not_accepted2 but not_accepted1:  3


In [23]:
for name in diff:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    print("Answers: ", ann1["answer"],"-", ann2["answer"])
    start = ann1["spans"][0]["start"]
    end = ann1["spans"][0]["end"]
    print(ann1["text"][start:end], "-> ", ann1['html'])
    print()

Answers:  reject - accept
Einklemmung ->  <b>C1261287</b>: Stenose: Der Zustand, dass eine anatomische Struktur über die normalen Maße hinaus eingeengt ist.

Answers:  reject - accept
Auslöser ->  <b>C0032930</b>: Niederschlagende Faktoren: Faktoren, die mit dem endgültigen Ausbruch einer Krankheit, eines Unfalls, einer Verhaltensreaktion oder einer Handlung zusammenhängen. Wenn mehrere Faktoren beteiligt sind, ist in der Regel ein Faktor wichtiger oder offensichtlicher als andere, und einer kann oft als "notwendig" angesehen werden. Beispiele hierfür sind die Exposition gegenüber einer bestimmten Krankheit, die Menge oder das Ausmaß eines infektiösen Organismus, einer Droge oder einer schädlichen Substanz usw.

Answers:  accept - reject
Magen Darm Virus ->  <b>C4082764</b>: Gastrointestinale Infektion: Wiederkehrende Infektionen des Magen-Darm-Trakts. [HPO:probinson]

Answers:  accept - reject
Schilddrüse ist in Ordnung ->  <b>C0040132</b>: Schilddrüse: Eine stark vaskularisierte endo

In [24]:
diff1 = [annotations1[name] for name in diff]
diff2 = [annotations2[name] for name in diff]

In [25]:
diff1[0]

{'text': 'Thread: [Einklemmung/Darmverschluss durch oder was anders? Ärzte ratlos]\nText: [Auch wenn ein Osteophat keine Verwachsungen fühlen konnte, könnten es Verwachsungsbeschwerden sein.\nDas ist oft unglaublich, wieviele und heftige Beschwerden Verwachsungen machen können.]\n',
 'spans': [{'start': 9, 'end': 20, 'label': 'Mention'}],
 'html': '<b>C1261287</b>: Stenose: Der Zustand, dass eine anatomische Struktur über die normalen Maße hinaus eingeengt ist.',
 'annotation_ids': [5652,
  6688,
  6492,
  5196,
  5563,
  5017,
  5330,
  6388,
  5037,
  4855,
  4985,
  6987,
  5075],
 'cui': 'C1261287',
 'meta': {'url': 'https://uts.nlm.nih.gov/uts/umls/concept/C1261287'},
 '_input_hash': 1591926120,
 '_task_hash': -1718244887,
 '_view_id': 'blocks',
 'answer': 'reject',
 '_timestamp': 1687772550,
 '_annotator_id': 'TLC-validation-second-round-alon',
 '_session_id': 'TLC-validation-second-round-alon'}

annotator agreement


In [20]:
# calculate kappa score given that we treat ignore as same with the other answer and treat both ignore as both rejected
a1, a2 = [], []
for name in annotations1:
    ann1 = annotations1[name]
    ann2 = annotations2[name]
    a1.append(ann1["answer"])
    a2.append(ann2["answer"])
    
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(a1, a2)

0.3867768595041322

In [21]:
from sklearn.metrics import classification_report
print(classification_report(a1, a2))

              precision    recall  f1-score   support

      accept       0.91      0.93      0.92        46
      reject       0.50      0.43      0.46         7

    accuracy                           0.87        53
   macro avg       0.71      0.68      0.69        53
weighted avg       0.86      0.87      0.86        53



In [133]:
print(classification_report(a2, a1))

              precision    recall  f1-score   support

      accept       0.89      0.91      0.90       574
      reject       0.79      0.75      0.77       258

    accuracy                           0.86       832
   macro avg       0.84      0.83      0.84       832
weighted avg       0.86      0.86      0.86       832



In [136]:

import collections

def parse_classification_report(clfreport):
    """
    Parse a sklearn classification report into a dict keyed by class name
    and containing a tuple (precision, recall, fscore, support) for each class
    """
    lines = clfreport.split('\n')
    # Remove empty lines
    lines = list(filter(lambda l: not len(l.strip()) == 0, lines))
    lines=[s for s in lines if 'accuracy' not in s]

    # Starts with a header, then score for each class and finally an average
    header = lines[0]
    cls_lines = lines[1:-1]
    avg_line = lines[-1]

    assert header.split() == ['precision', 'recall', 'f1-score', 'support']
    assert avg_line.split()[1] == 'avg'

    # We cannot simply use split because class names can have spaces. So instead
    # figure the width of the class field by looking at the indentation of the
    # precision header
    cls_field_width = len(header) - len(header.lstrip())
    # Now, collect all the class names and score in a dict
    def parse_line(l):
        """Parse a line of classification_report"""
        cls_name = l[:cls_field_width].strip()
        precision, recall, fscore, support = l[cls_field_width:].split()
        precision = float(precision)
        recall = float(recall)
        fscore = float(fscore)
        support = int(support)
        return (cls_name, precision, recall, fscore, support)

    data = collections.OrderedDict()
    for l in cls_lines:
        ret = parse_line(l)
        cls_name = ret[0]
        scores = ret[1:]
        data[cls_name] = scores

    # average
    data['avg'] = parse_line(avg_line)[1:]

    return data

def report_to_latex_table(data):
    avg_split = False
    out = ""
    out += "\\begin{table}\n"
    out += "\\caption{Latex Table from Classification Report}\n"
    out += "\\label{table:classification:report}\n"
    out += "\\centering\n"
    out += "\\begin{tabular}{c | c c c r}\n"
    out += "Class & Precision & Recall & F-score & Support\\\\\n"
    out += "\midrule\n"
    for cls, scores in data.items():
        if 'micro' in cls:
            out += "\\midrule\n"
        out += cls + " & " + " & ".join([str(s) for s in scores])
        out += "\\\\\n"
    out += "\\end{tabular}\n"
    out += "\\end{table}"
    return out

print(report_to_latex_table(parse_classification_report(classification_report(a1, a2))))

\begin{table}
\caption{Latex Table from Classification Report}
\label{table:classification:report}
\centering
\begin{tabular}{c | c c c r}
Class & Precision & Recall & F-score & Support\\
\midrule
accept & 0.91 & 0.89 & 0.9 & 587\\
reject & 0.75 & 0.79 & 0.77 & 245\\
macro avg & 0.83 & 0.84 & 0.84 & 832\\
avg & 0.86 & 0.86 & 0.86 & 832\\
\end{tabular}
\end{table}


'              precision    recall  f1-score   support\n\n      accept       0.91      0.89      0.90       587\n      reject       0.75      0.79      0.77       245\n\n    accuracy                           0.86       832\n   macro avg       0.83      0.84      0.84       832\nweighted avg       0.86      0.86      0.86       832\n'